In [1]:
!pip install auto-gptq
!pip install optimum
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install accelerate
!pip install torch==2.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 9.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of huggingface-hub to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of huggingface-hub to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 33.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Load libraries

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
import transformers
import torch

# Load model

In [6]:
# Model name
model_name = "meta-llama/Llama-2-7b-hf"

# Config to load model in 4bits
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Initiate model to compare effect of fine tuning
original_model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",
                                             quantization_config=bnb_config,
                                             trust_remote_code=True,
                                             revision="main",
                                             is_decoder=True)

# Model for fine tuning
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",
                                             quantization_config=bnb_config,
                                             trust_remote_code=True,
                                             revision="main",
                                             is_decoder=True)

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
def print_number_of_trainable_model_parameters(model):
  trainable_model_params = 0
  all_model_params = 0
  for _, param in model.named_parameters():
    all_model_params += param.numel()
    if param.requires_grad:
      trainable_model_params += param.numel()
  return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(model))

trainable model parameters: 262410240
all model parameters: 3500412928
percentage of trainable model parameters: 7.50%


# Load dataset

In [8]:
dataset = load_dataset("cnn_dailymail", "3.0.0")

dataset

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

# Load tokenizer

In [9]:
# Load tokenizer used in the model
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

# Test the model with zero shot inferencing

In [10]:
index = 400

article = dataset["test"][index]["article"]
highlights = dataset["test"][index]["highlights"]

prompt = f"""
Summarize the following article.

Article: {article}

Highlights:
"""

inputs = tokenizer(prompt, return_tensors="pt")
output = original_model.generate(
    inputs["input_ids"],
    max_new_tokens=100,
)

tokenized_output = tokenizer.decode(output[0], skip_special_tokens=True)

dash_line = "-".join("" for x in range(100))
print(dash_line)
print(f'INPUT_PROMPT:¥n{prompt}')
print(dash_line)
print(f'BASELINE HIGHLIGHTS:¥n{highlights}')
print(dash_line)
print(f'ARTICLE_SUMMARY:¥n{tokenized_output}')

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1460: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


---------------------------------------------------------------------------------------------------
INPUT_PROMPT:¥n
Summarize the following article.

Article: (CNN)The mass killings of Armenians in the Ottoman Empire, which began 100 years ago Friday, is said by some scholars and others to have been the first genocide of the 20th century, even though the word "genocide" did not exist at the time. The issue of whether to call the killings a genocide is emotional, both for Armenians, who are descended from those killed, and for Turks, the heirs to the Ottomans. For both groups, the question touches as much on national identity as on historical facts. Some Armenians feel their nationhood cannot be fully recognized unless the truth of what happened to their forebears is acknowledged. Some Turks still view the Armenians as having been a threat to the Ottoman Empire in a time of war, and say many people of various ethnicities -- including Turks -- were killed in the chaos of war. In addition

# Tokenizer function

In [11]:
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

In [12]:
dataset = dataset.filter(lambda example, index: index % 400 == 0, with_indices=True)

Filter:   0%|          | 0/287113 [00:00<?, ? examples/s]

Filter:   0%|          | 0/13368 [00:00<?, ? examples/s]

Filter:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [13]:
def tokenize_function(example):
  start_prompt = "Summarize the following article. Article: ¥n¥n"
  end_prompt = "¥n¥nHighlights:"

  prompt = [start_prompt + article + end_prompt for article in example["article"]]
  example['input_ids'] = tokenizer(prompt, padding="max_length", max_length=1058, truncation=True, return_tensors="pt").input_ids
  example['labels'] = tokenizer(example["highlights"], padding="max_length", max_length=85, truncation=True, return_tensors="pt").input_ids

  return example

tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/718 [00:00<?, ? examples/s]

Map:   0%|          | 0/34 [00:00<?, ? examples/s]

Map:   0%|          | 0/29 [00:00<?, ? examples/s]

In [14]:
tokenized_dataset = tokenized_dataset.remove_columns(["article", "highlights", "id"])


In [15]:
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_dataset['train'].shape}")
print(f"Validation: {tokenized_dataset['validation'].shape}")
print(f"Test: {tokenized_dataset['test'].shape}")

print(tokenized_dataset)

Shapes of the datasets:
Training: (718, 2)
Validation: (34, 2)
Test: (29, 2)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 718
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 34
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 29
    })
})


# Prepare model for training

In [16]:
model.train()

model.gradient_checkpointing_enable()

model = prepare_model_for_kbit_training(model)

In [17]:
# LoRA config
config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["o_proj", "q_proj", "k_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# LoRA trainanle version of model
model = get_peft_model(model, config)

model.print_trainable_parameters()

trainable params: 8,388,608 || all params: 6,746,812,416 || trainable%: 0.12433438908285782


# Fine-tuning Model


In [18]:
import time

# hyperparameters
lr = 2e-4
batch_size = 2
epochs = 2

output_dir = f"./article-summary-training-{str(int(time.time()))}"

training_args = transformers.TrainingArguments(
    output_dir=output_dir,
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs,
    weight_decay=0.01,
    logging_strategy="epoch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    gradient_accumulation_steps=4,
    warmup_steps=2,
    fp16=True,
    optim="paged_adamw_8bit"
)


In [19]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

model.config.user_cache = False
trainer.train()

model.config.use_cache = True

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be

Epoch,Training Loss,Validation Loss
0,1.756800,1.693011
1,1.671400,1.691990


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:168: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:168: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


# Evaluation

In [20]:
index = 28
article = dataset["test"][index]["article"]
highlights = dataset["test"][index]["highlights"]

prompt = f"""
Summarize the following article.

{article}

Highlights:
"""

input_ids = tokenizer(prompt, return_tensors="pt").input_ids

original_model_outputs = original_model.generate(input_ids=input_ids, max_new_tokens=100)
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

instruct_model_outputs = model.generate(input_ids=input_ids, max_new_tokens=100)
instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{highlights}')
print(dash_line)
print(f'ORIGINAL MODEL:\n{original_model_text_output}')
print(dash_line)
print(f'INSTRUCT MODEL:\n{instruct_model_text_output}')

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1460: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
Juanette Cullum, 48, also allegedly stole two laptops and many toiletries .
She was arrested after a witness saw her stealing property, prosecutors said .
Officials said Cullen admitted to stealing from American Airline planes for the last three years .
Was charged with grand larceny and criminal possession of stolen property .
---------------------------------------------------------------------------------------------------
ORIGINAL MODEL:

Summarize the following article.

JFK airplane cleaner Juanette Cullum, 48, allegedly stole almost $15,000 items from American Airline planes over the last three years . An airplane cleaner at John F. Kennedy International Airport stole almost $15,000 worth of items, including more than 1,400 unopened miniature bottles of liquor, according to police. Juanette Cullum, 48, was arrested after a witness allegedly saw her stealing

In [21]:
# save locally
model.save_pretrained("llama-article-summarize-model", config=config)

# push to the hub
model.push_to_hub("llama-article-summarize-model", config=config)

# reload
model = model.from_pretrained("username/my-awesome-model")

/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:168: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/1.08G [00:00<?, ?B/s]

TypeError: PeftModel.from_pretrained() missing 1 required positional argument: 'model_id'